In [4]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import os
os.chdir('/content/drive/MyDrive/365 frontier ai/day 3/')

In [6]:
#read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [7]:
len(words)

32033

In [8]:
#build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [9]:
#build the dataset

block_size = 3 #context length
X, Y = [], []
for w in words[:5]:
  print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] #crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [10]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [15]:
C = torch.randn((27,2))

In [14]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [16]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [19]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape

torch.Size([32, 6])

In [22]:
torch.cat(torch.unbind(emb, 1), 1).shape

torch.Size([32, 6])

In [24]:
a = torch.arange(18)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [25]:
a.shape

torch.Size([18])

In [28]:
a.view(3,3,2)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])

In [29]:
a.storage()

/tmp/ipython-input-214256462.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a.storage()


 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [30]:
emb.view(32,6)

tensor([[ 0.6105,  0.0479,  0.6105,  0.0479,  0.6105,  0.0479],
        [ 0.6105,  0.0479,  0.6105,  0.0479,  1.5843, -1.7457],
        [ 0.6105,  0.0479,  1.5843, -1.7457,  0.5174,  0.7630],
        [ 1.5843, -1.7457,  0.5174,  0.7630,  0.5174,  0.7630],
        [ 0.5174,  0.7630,  0.5174,  0.7630,  1.7713,  1.1415],
        [ 0.6105,  0.0479,  0.6105,  0.0479,  0.6105,  0.0479],
        [ 0.6105,  0.0479,  0.6105,  0.0479, -0.5474, -0.3600],
        [ 0.6105,  0.0479, -0.5474, -0.3600, -0.0622, -0.5405],
        [-0.5474, -0.3600, -0.0622, -0.5405, -0.1467,  0.6636],
        [-0.0622, -0.5405, -0.1467,  0.6636,  1.5213,  0.0249],
        [-0.1467,  0.6636,  1.5213,  0.0249, -0.1467,  0.6636],
        [ 1.5213,  0.0249, -0.1467,  0.6636,  1.7713,  1.1415],
        [ 0.6105,  0.0479,  0.6105,  0.0479,  0.6105,  0.0479],
        [ 0.6105,  0.0479,  0.6105,  0.0479,  1.7713,  1.1415],
        [ 0.6105,  0.0479,  1.7713,  1.1415,  1.5213,  0.0249],
        [ 1.7713,  1.1415,  1.5213,  0.0

In [37]:
h = torch.tanh(emb.view(-1,6) @ W1 + b1)

In [40]:
h.shape

torch.Size([32, 100])

In [41]:
W2 = torch.randn(100,27)
b2 = torch.randn(27)

In [44]:
logits = h @ W2 + b2

In [45]:
logits.shape

torch.Size([32, 27])

In [46]:
counts = logits.exp()

In [49]:
prob = counts /counts.sum(1, keepdims=True)

In [52]:
prob.shape

torch.Size([32, 27])

In [57]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(16.5928)

In [54]:
torch.arange(32)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [53]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [58]:
####---------cleaner version------------#####

In [59]:
X.shape, Y.shape #dataset

(torch.Size([32, 3]), torch.Size([32]))

In [62]:
g = torch.Generator().manual_seed(2147483647)   #for reproducibility
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6,100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100,27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [63]:
sum(p.nelement() for p in parameters) #number of parameters in total

3481

In [65]:
emb = C[X] #(32,3,2)
h = torch.tanh(emb.view(-1,6)@ W1 + b1) #(32,100)
logits = h @ W2 + b2 #(32,27)
counts = logits.exp()
prob = counts / counts.sum(1, keepdims= True)
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.7697)

In [66]:
F.cross_entropy(logits, Y)

tensor(17.7697)